<div style="background: linear-gradient(135deg, #092322ff 0%, #97e8ebff 100%); 
            padding: 30px; 
            border-radius: 15px; 
            color: white; 
            font-family: 'Segoe UI', Arial, sans-serif;
            box-shadow: 0 8px 16px rgba(126, 109, 109, 0.2);
            max-width: 1100px;  /* Add this - limits the width */
            margin: 0 auto;">
    <h1 style="margin: 0 0 20px 0; font-size: 2.5em; text-align: center;">
        🎮 Quantum Gate Guessing Game
    </h1>
    <div style="background: rgba(255,255,255,0.1); padding: 20px; border-radius: 10px; backdrop-filter: blur(10px);">
        <h2 style="margin-top: 0; color: #ffd700;">📖 How to Play</h2>
        <p style="font-size: 1.1em; line-height: 1.6;">
            <strong>Objective:</strong> Identify the secret quantum gate through experimentation.
        </p>
        <h3 style="color: #ffd700; margin-top: 20px;">🎯 Game Flow:</h3>
        <ol style="font-size: 1.05em; line-height: 1.8;">
            <li><strong>Secret Selection:</strong> The game randomly chooses one of 12 quantum gates</li>
            <li><strong>Your Tools:</strong> You have 6 probe states to test with:
                <ul>
                    <li>|0⟩, |1⟩ (computational basis)</li>
                    <li>|+⟩, |−⟩ (superposition states)</li>
                    <li>|i⟩, |−i⟩ (circular basis states)</li>
                </ul>
            </li>
            <li><strong>Three Rounds:</strong> In each round, choose a probe state, send it through the secret gate, and observe the output</li>
            <li><strong>Make Your Guess:</strong> After 3 probes, identify which gate it was!</li>
        </ol>
        <h3 style="color: #ffd700; margin-top: 20px;">💡 Strategy Tip:</h3>
        <p style="font-size: 1.05em; line-height: 1.6; background: rgba(255,255,255,0.1); padding: 15px; border-radius: 8px;">
            Different gates affect different probe states uniquely. For example:<br>
            • X gate flips |0⟩↔|1⟩ but leaves |+⟩ unchanged<br>
            • Z gate flips |+⟩↔|−⟩ but leaves |0⟩ unchanged<br>
            • Phase gates (S, T) add phases that show clearly in superposition states
        </p>
    </div>
</div>


### Import Dependencies
#### We need random for selecting the secret gate and math.pi for rotation angles

In [22]:
import random
from math import pi
from IPython.display import display, HTML

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


###  Define Available Probe States
##### These are the 6 quantum states you can use to probe the secret gate.
##### Each represents a different single-qubit quantum state:
##### - Computational basis: |0⟩, |1⟩
##### - Superposition states: |+⟩, |−⟩ (equal superposition with ± phase)
##### - Circular basis: |i⟩, |−i⟩ (superposition with imaginary phase)

In [9]:
PROBE_STATES = {
    1: "|0>",
    2: "|1>",
    3: "|+> = (|0> + |1>)/√2",
    4: "|−> = (|0> − |1>)/√2",
    5: "|i> = (|0> + i|1>)/√2",
    6: "|-i> = (|0> - i|1>)/√2"
}

print("Available probe states:")
for k, v in PROBE_STATES.items():
    print(f"  {k}: {v}")

Available probe states:
  1: |0>
  2: |1>
  3: |+> = (|0> + |1>)/√2
  4: |−> = (|0> − |1>)/√2
  5: |i> = (|0> + i|1>)/√2
  6: |-i> = (|0> - i|1>)/√2


####  Define the 12 Secret Gates
##### These are the quantum gates that could be the secret gate.
##### The list includes:
##### - Pauli gates: I, X, Y, Z (fundamental single-qubit gates)
##### - Hadamard: H (creates superposition)
##### - Phase gates: S, S†, T, T† (add phase rotations)
##### - Rotation gates: RX, RY, RZ (continuous rotations around Bloch sphere axes)

In [10]:
UNITARIES = [
    "I", "X", "Y", "Z",
    "H",
    "S", "Sdg",
    "T", "Tdg",
    "RX(pi/2)", "RY(pi/2)", "RZ(pi/4)"
]

print(f"Total gates in the game: {len(UNITARIES)}")
print("Gates:", ", ".join(UNITARIES))

Total gates in the game: 12
Gates: I, X, Y, Z, H, S, Sdg, T, Tdg, RX(pi/2), RY(pi/2), RZ(pi/4)


### Gate Descriptions
#### Human-readable descriptions of what each gate does.
#### This helps players understand the physical meaning of each transformation.

In [12]:
GATE_INFO = {
    "I": "Identity (does nothing).",
    "X": "Bit flip (|0> ↔ |1>).",
    "Y": "Bit+phase flip (|0>→i|1>, |1>→−i|0>).",
    "Z": "Phase flip (|1> gets a - sign).",
    "H": "Hadamard (creates ± superposition).",
    "S": "Phase gate (adds +i phase to |1>).",
    "Sdg": "Inverse phase (adds −i to |1>).",
    "T": "π/8 gate (adds e^{iπ/4} phase).",
    "Tdg": "Inverse π/8 (adds e^{-iπ/4} phase).",
    "RX(pi/2)": "Rotation around X-axis by π/2.",
    "RY(pi/2)": "Rotation around Y-axis by π/2.",
    "RZ(pi/4)": "Rotation around Z-axis by π/4."
}

print("Gate descriptions loaded:")
for gate, desc in GATE_INFO.items():
    print(f"  {gate:12} → {desc}")

Gate descriptions loaded:
  I            → Identity (does nothing).
  X            → Bit flip (|0> ↔ |1>).
  Y            → Bit+phase flip (|0>→i|1>, |1>→−i|0>).
  Z            → Phase flip (|1> gets a - sign).
  H            → Hadamard (creates ± superposition).
  S            → Phase gate (adds +i phase to |1>).
  Sdg          → Inverse phase (adds −i to |1>).
  T            → π/8 gate (adds e^{iπ/4} phase).
  Tdg          → Inverse π/8 (adds e^{-iπ/4} phase).
  RX(pi/2)     → Rotation around X-axis by π/2.
  RY(pi/2)     → Rotation around Y-axis by π/2.
  RZ(pi/4)     → Rotation around Z-axis by π/4.


### Initialize Transformation Dictionary
#### This dictionary will store all possible transformations.
#### Key: (probe_number, gate_name)
#### Value: resulting quantum state after applying the gate
#### We'll populate this using a helper function for cleaner code.

In [13]:
RESULT = {}

def add(probe_num, gate_name, result_state):
    """Helper function to add a transformation to the lookup table."""
    RESULT[(probe_num, gate_name)] = result_state

print("Transformation dictionary initialized")
print(f"Total entries to be added: {len(PROBE_STATES)} probes × {len(UNITARIES)} gates = {len(PROBE_STATES) * len(UNITARIES)}")


Transformation dictionary initialized
Total entries to be added: 6 probes × 12 gates = 72


### Define |0⟩ Transformations (Probe 1)
#### How each gate transforms the |0⟩ state.
#### This is the simplest case - many gates leave |0⟩ unchanged or apply simple operations.

In [14]:
add(1,"I","|0>")                      # Identity: no change
add(1,"X","|1>")                      # X flips to |1⟩
add(1,"Y","i|1>")                     # Y flips and adds phase
add(1,"Z","|0>")                      # Z doesn't affect |0⟩
add(1,"H","(|0> + |1>)/√2")           # H creates superposition
add(1,"S","|0>")                      # S doesn't affect |0⟩
add(1,"Sdg","|0>")                    # S† doesn't affect |0⟩
add(1,"T","|0>")                      # T doesn't affect |0⟩
add(1,"Tdg","|0>")                    # T† doesn't affect |0⟩
add(1,"RX(pi/2)","(|0> − i|1>)/√2")   # RX rotates around X
add(1,"RY(pi/2)","(|0> + |1>)/√2")    # RY rotates around Y
add(1,"RZ(pi/4)","|0>")               # RZ doesn't affect |0⟩

print("✓ |0⟩ transformations defined (12 gates)")

✓ |0⟩ transformations defined (12 gates)


### Define |1⟩ Transformations (Probe 2)
#### How each gate transforms the |1⟩ state.
#### Phase gates have more visible effects on |1⟩ compared to |0⟩.

In [15]:
add(2,"I","|1>")                      # Identity: no change
add(2,"X","|0>")                      # X flips to |0⟩
add(2,"Y","−i|0>")                    # Y flips with negative i phase
add(2,"Z","−|1>")                     # Z adds minus sign
add(2,"H","(|0> − |1>)/√2")           # H creates |−⟩ superposition
add(2,"S","i|1>")                     # S adds +i phase
add(2,"Sdg","−i|1>")                  # S† adds −i phase
add(2,"T","e^{iπ/4}|1>")              # T adds e^(iπ/4) phase
add(2,"Tdg","e^{-iπ/4}|1>")           # T† adds e^(−iπ/4) phase
add(2,"RX(pi/2)","(|1> − i|0>)/√2")   # RX rotates around X
add(2,"RY(pi/2)","(|1> − |0>)/√2")    # RY rotates around Y
add(2,"RZ(pi/4)","e^{iπ/4}|1>")       # RZ adds phase

print("✓ |1⟩ transformations defined (12 gates)")

✓ |1⟩ transformations defined (12 gates)


### Define |+⟩ Transformations (Probe 3)
#### How each gate transforms the |+⟩ = (|0⟩ + |1⟩)/√2 state.
#### This superposition state reveals different behavior for many gates.

In [16]:
add(3,"I","|+>")                              # Identity: no change
add(3,"X","|+>")                              # X leaves |+⟩ unchanged!
add(3,"Y","i|−>")                             # Y converts to |−⟩ with phase
add(3,"Z","|−>")                              # Z flips to |−⟩
add(3,"H","|0>")                              # H converts back to |0⟩
add(3,"S","|i>")                              # S converts to |i⟩
add(3,"Sdg","|-i>")                           # S† converts to |−i⟩
add(3,"T","(|0> + e^{iπ/4}|1>)/√2")           # T adds phase to |1⟩ component
add(3,"Tdg","(|0> + e^{-iπ/4}|1>)/√2")        # T† adds opposite phase
add(3,"RX(pi/2)","|0>")                       # RX collapses to |0⟩
add(3,"RY(pi/2)","(|0> + i|1>)/√2 = |i>")     # RY creates |i⟩
add(3,"RZ(pi/4)","(|0> + e^{iπ/4}|1>)/√2")    # RZ adds phase

print("✓ |+⟩ transformations defined (12 gates)")

✓ |+⟩ transformations defined (12 gates)


### Define |−⟩ Transformations (Probe 4)
#### How each gate transforms the |−⟩ = (|0⟩ − |1⟩)/√2 state.
#### This is complementary to |+⟩ and helps distinguish certain gates.

In [17]:
add(4,"I","|−>")                              # Identity: no change
add(4,"X","|−>")                              # X leaves |−⟩ unchanged!
add(4,"Y","−i|+>")                            # Y converts to |+⟩ with phase
add(4,"Z","|+>")                              # Z flips to |+⟩
add(4,"H","|1>")                              # H converts to |1⟩
add(4,"S","(|0> - i|1>)/√2 = |-i>")           # S creates |−i⟩
add(4,"Sdg","(|0> + i|1>)/√2 = |i>")          # S† creates |i⟩
add(4,"T","(|0> − e^{iπ/4}|1>)/√2")           # T adds phase
add(4,"Tdg","(|0> − e^{-iπ/4}|1>)/√2")        # T† adds opposite phase
add(4,"RX(pi/2)","|1>")                       # RX collapses to |1⟩
add(4,"RY(pi/2)","(|0> − i|1>)/√2")           # RY creates superposition
add(4,"RZ(pi/4)","(|0> − e^{iπ/4}|1>)/√2")    # RZ adds phase

print("✓ |−⟩ transformations defined (12 gates)")

✓ |−⟩ transformations defined (12 gates)


### Define |i⟩ Transformations (Probe 5)
#### How each gate transforms the |i⟩ = (|0⟩ + i|1⟩)/√2 state.
#### This circular basis state is useful for distinguishing phase gates.

In [18]:
add(5,"I","|i>")                              # Identity: no change
add(5,"X","i|0>")                             # X creates phase + |0⟩
add(5,"Y","|0>")                              # Y collapses to |0⟩!
add(5,"Z","(|0> − i|1>)/√2")                  # Z converts to |−i⟩
add(5,"H","(1−i)|0>/2 + (1+i)|1>/2")          # H creates complex superposition
add(5,"S","i|i>")                             # S adds global phase
add(5,"Sdg","−i|i>")                          # S† adds opposite global phase
add(5,"T","e^{iπ/8}|i>")                      # T adds phase
add(5,"Tdg","e^{-iπ/8}|i>")                   # T† adds opposite phase
add(5,"RX(pi/2)","complex superposition")     # RX creates complex state
add(5,"RY(pi/2)","complex superposition")     # RY creates complex state
add(5,"RZ(pi/4)","e^{iπ/4}|i>")               # RZ adds phase

print("✓ |i⟩ transformations defined (12 gates)")

✓ |i⟩ transformations defined (12 gates)


### Define |−i⟩ Transformations (Probe 6)
#### How each gate transforms the |−i⟩ = (|0⟩ − i|1⟩)/√2 state.
#### Complementary to |i⟩ for complete basis coverage.

In [19]:
add(6,"I","|-i>")                             # Identity: no change
add(6,"X","−i|0>")                            # X creates phase + |0⟩
add(6,"Y","−|0>")                             # Y collapses to −|0⟩
add(6,"Z","(|0> + i|1>)/√2")                  # Z converts to |i⟩
add(6,"H","(1+i)|0>/2 + (1−i)|1>/2")          # H creates complex superposition
add(6,"S","−i|-i>")                           # S adds global phase
add(6,"Sdg","i|-i>")                          # S† adds opposite global phase
add(6,"T","e^{iπ/8}|-i>")                     # T adds phase
add(6,"Tdg","e^{-iπ/8}|-i>")                  # T† adds opposite phase
add(6,"RX(pi/2)","complex superposition")     # RX creates complex state
add(6,"RY(pi/2)","complex superposition")     # RY creates complex state
add(6,"RZ(pi/4)","e^{iπ/4}|-i>")              # RZ adds phase

print("✓ |−i⟩ transformations defined (12 gates)")
print(f"\nTotal transformations in lookup table: {len(RESULT)}")

✓ |−i⟩ transformations defined (12 gates)

Total transformations in lookup table: 72


### Main Game Function
#### This function implements the interactive game loop:
#### 1. Randomly select a secret gate
#### 2. Display available gates and probes
#### 3. Allow 3 probe attempts
#### 4. Check the final guess
#### The game teaches quantum mechanics through experimentation!

In [20]:
def play_game():
    """
    Interactive quantum gate guessing game.
    
    Players probe a secret gate with different quantum states and must
    identify which gate it is based on the output states.
    """
    # Randomly select the secret gate
    secret = random.choice(UNITARIES)

    print("=" * 60)
    print("   QUANTUM GATE GUESSING GAME")
    print("=" * 60)
    print("\nSecret gate is one of:\n")
    
    # Show all possible gates with descriptions
    for g in UNITARIES:
        print(f" {g:<12} - {GATE_INFO[g]}")
    print()

    # Show available probe states
    print("\nAvailable Probes:")
    for k, v in PROBE_STATES.items():
        print(f"  {k} = {v}")
    
    print("\n" + "=" * 60)
    print("You get 3 probe attempts to identify the secret gate!")
    print("=" * 60 + "\n")

    # Three rounds of probing
    for round_num in [1, 2, 3]:
        print(f"\n{'─' * 60}")
        print(f"ROUND {round_num}/3")
        print('─' * 60)
        
        # Get probe choice from user
        probe_num = int(input("Choose probe number (1-6): "))
        
        # Show the input probe state
        print("\n📥 Input Probe State:")
        print(f"   {PROBE_STATES[probe_num]}")
        
        # Show the output after applying secret gate
        print("\n📤 Output State (after secret gate):")
        print(f"   {RESULT[(probe_num, secret)]}")
        print()

    # Final guess
    print("\n" + "=" * 60)
    print("TIME TO GUESS!")
    print("=" * 60)
    guess = input("\nYour final guess (exact gate name): ").strip()
    
    # Check answer
    print("\n" + "=" * 60)
    if guess == secret:
        print("🎉 CORRECT! Well done!")
        print(f"The secret gate was indeed: {secret}")
        print(f"Description: {GATE_INFO[secret]}")
    else:
        print("❌ Wrong guess!")
        print(f"Your guess: {guess}")
        print(f"The secret gate was: {secret}")
        print(f"Description: {GATE_INFO[secret]}")
    print("=" * 60)
    
    print("\n✨ Game finished. Run play_game() again to play another round!")

print("\n" + "=" * 60)
print("Setup complete! Ready to play.")
print("=" * 60)
print("\n▶ Type: play_game()")
print("  to start the game!\n")



Setup complete! Ready to play.

▶ Type: play_game()
  to start the game!



### Demo

In [24]:
play_game()

   QUANTUM GATE GUESSING GAME

Secret gate is one of:

 I            - Identity (does nothing).
 X            - Bit flip (|0> ↔ |1>).
 Y            - Bit+phase flip (|0>→i|1>, |1>→−i|0>).
 Z            - Phase flip (|1> gets a - sign).
 H            - Hadamard (creates ± superposition).
 S            - Phase gate (adds +i phase to |1>).
 Sdg          - Inverse phase (adds −i to |1>).
 T            - π/8 gate (adds e^{iπ/4} phase).
 Tdg          - Inverse π/8 (adds e^{-iπ/4} phase).
 RX(pi/2)     - Rotation around X-axis by π/2.
 RY(pi/2)     - Rotation around Y-axis by π/2.
 RZ(pi/4)     - Rotation around Z-axis by π/4.


Available Probes:
  1 = |0>
  2 = |1>
  3 = |+> = (|0> + |1>)/√2
  4 = |−> = (|0> − |1>)/√2
  5 = |i> = (|0> + i|1>)/√2
  6 = |-i> = (|0> - i|1>)/√2

You get 3 probe attempts to identify the secret gate!


────────────────────────────────────────────────────────────
ROUND 1/3
────────────────────────────────────────────────────────────

📥 Input Probe State:
   |+> = (